# **Part 1: Building Model & Calculating Accuracy**


**1. Import Data**

In [1]:
pip install python-mnist

In [2]:
from mnist import MNIST

In [11]:
data = MNIST(path='/content/drive/MyDrive/Projects/ARS/data', return_type='numpy')
data.select_emnist('letters')
X, y = data.load_training()

In [12]:
X.shape, y.shape

((124800, 784), (124800,))

In [13]:
X = X.reshape(124800, 28, 28)
y = y.reshape(124800, 1)

In [6]:
# list(y) -> y ranges from 1 to 26

In [14]:
y = y-1

In [ ]:
# list(y) -> now y rangers from 0 to 25

**2. Train - Test split**

In [10]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 50)

In [16]:
# (0,255) --> (0,1)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [17]:
y_train, y_test

(array([[ 2],
        [25],
        [14],
        ...,
        [12],
        [ 3],
        [12]], dtype=uint8), array([[ 9],
        [16],
        [10],
        ...,
        [ 5],
        [14],
        [25]], dtype=uint8))

In [18]:
# Converting integer into one hot vector (binary class matrix)
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, num_classes = 26)
y_test = np_utils.to_categorical(y_test, num_classes = 26)

In [19]:
y_train, y_test

(array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32))

**3. Defining the Model**

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

model = Sequential()
model.add(Flatten(input_shape = (28,28)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2)) # preventing overfitting
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(26, activation='softmax'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 26)                13338     
Total params: 677,914
Trainable params: 677,914
Non-trainable params: 0
__________________________________________________

In [22]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

**4. Calcuating Accuracy**

In [24]:
# before training
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]
print("Before training, test accuracy is", accuracy)

780/780 [==============================] - 2s 3ms/step - loss: 3.2836 - accuracy: 0.0367
Before training, test accuracy is 3.669871762394905


In [25]:
# training the model
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'ars_model.h5', verbose=1, save_best_only = True)
model.fit(X_train, y_train, batch_size = 128, epochs= 10, validation_split = 0.2, 
          callbacks=[checkpointer], verbose=1, shuffle=True)

model.load_weights('ars_model.h5')

Epoch 1/10
624/624 [==============================] - 11s 18ms/step - loss: 0.8064 - accuracy: 0.7554 - val_loss: 0.4255 - val_accuracy: 0.8660

Epoch 00001: val_loss improved from inf to 0.42554, saving model to ars_model.h5
Epoch 2/10
624/624 [==============================] - 10s 17ms/step - loss: 0.4225 - accuracy: 0.8660 - val_loss: 0.3425 - val_accuracy: 0.8908

Epoch 00002: val_loss improved from 0.42554 to 0.34251, saving model to ars_model.h5
Epoch 3/10
624/624 [==============================] - 11s 17ms/step - loss: 0.3408 - accuracy: 0.8884 - val_loss: 0.3109 - val_accuracy: 0.8995

Epoch 00003: val_loss improved from 0.34251 to 0.31087, saving model to ars_model.h5
Epoch 4/10
624/624 [==============================] - 11s 17ms/step - loss: 0.2937 - accuracy: 0.9012 - val_loss: 0.2872 - val_accuracy: 0.9064

Epoch 00004: val_loss improved from 0.31087 to 0.28716, saving model to ars_model.h5
Epoch 5/10
624/624 [==============================] - 11s 17ms/step - loss: 0.2651 -

In [27]:
model.load_weights('ars_model.h5')

In [28]:
# after training
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]

print("After training, Test accuracy is ", accuracy)

780/780 [==============================] - 3s 4ms/step - loss: 0.2790 - accuracy: 0.9116
After training, Test accuracy is  91.15785360336304


# **Part 2: Alphabet Recognition**

In [29]:
from keras.models import load_model

In [30]:
model = load_model('ars_model.h5')

In [31]:
letters = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i', 9:'j', 10:'k', 11:'l', 
          12:'m', 13:'n', 14:'o', 15:'p', 16:'q', 17:'r', 18:'s', 19:'t', 20:'u', 21:'v', 22:'w', 
          23:'x', 24:'y', 25:'z', 26:''}

In [32]:
# defining pen color (blue) in hsv format

import numpy as np

blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 25, 25])

In [33]:
# Smoothning the image
kernel = np.ones((5,5), np.uint8)

In [34]:
# defining blackboard
blackboard = np.zeros((480,640, 3), dtype=np.uint8)
alphabet = np.zeros((200,200,3), dtype=np.uint8)

In [35]:
# deques (Double ended queue) is used to store alphabet drawn on screen
from collections import deque
points = deque(maxlen = 512)

**Opening Camera to recognize alphabet**

In [42]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame=cap.read()
    

    if cv2.waitKey(1) == 13: #if we press ENTER
      break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: ignored